In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.realpath('..'))

In [2]:
import numpy as np
from scipy.spatial.transform import Rotation

from numpy.typing import ArrayLike
from typing import Union

from tracking_v2.motion import ConstantVelocityModel

In [3]:
def print_rot(r: Rotation):
    roll, pitch, yaw = r.as_euler('xyz', degrees=True)
    print(f"yaw: {yaw:.2f}, pitch: {pitch:.2f}, roll: {roll:.2f}")

In [4]:
class Transform(object):
    pass

class Frame(object):
    """Global frame is one with origin (0,0,0) and unity rotation. Local frame
    is anything else."""
    
    # Choose right vector (arbitrary but stable)
    # use world +Y as up, so right = up × forward
    UP = np.asarray([0, 0, 1])

    def __init__(self, origin: ArrayLike, orientation: Union[ArrayLike, Rotation]):
        self.origin = np.asarray(origin)
        
        if isinstance(orientation, Rotation):
            self.rotation = orientation
        else:
            self.rotation = Frame.to_rotation(orientation)

    def __repr__(self) -> str:
        r, p, y = self.rotation.as_euler('xyz', degrees=True)
        return f"Frame(origin={self.origin.tolist()}, rotation=({y:.2f}, {p:.2f}, {r:.2f}))"

    def from_local_direction(self, local_direction: ArrayLike) -> 'Frame':
        """Global frame of the direction relative to this frame"""
        local_rotation = Frame.to_rotation(local_direction)
        return Frame(self.origin, self.rotation * local_rotation)
    
    def to_local_vec(self, global_vec: ArrayLike) -> np.ndarray:
        """Express global vector in frame"""
        global_vec = np.asarray(global_vec).flatten()
        
        if len(global_vec) == 3:
            return self.rotation.apply(global_vec, inverse=True) - self.origin
        elif len(global_vec) == 6:
            local_vec = np.zeros(6)           
            local_vec[:3] = self.rotation.apply(global_vec[:3], inverse=True) - self.origin
            local_vec[3:] = self.rotation.apply(global_vec[3:], inverse=True)
            return local_vec
        else:
            raise Exception(f"Unsupported vector shape {global_cov.shape}")

    def to_global_vec(self, local_vec: ArrayLike) -> np.ndarray:
        local_vec = np.asarray(local_vec)
        assert local_vec.shape[-1] == 3

        global_vec = self.rotation.apply(local_vec)
        return global_vec + self.origin
    
    def to_local_cov(self, global_cov: ArrayLike) -> np.ndarray:
        """Express global covariance matrix in frame"""
        r = self.rotation.inv().as_matrix()
        
        global_cov = np.asarray(global_cov)
        if global_cov.shape == (3, 3):
            return r @ global_cov @ r.T
        elif global_cov.shape == (6, 6):
            ret = np.zeros_like(global_cov)
            ret[:3, :3] = r @ global_cov[:3, :3] @ r.T
            ret[3:, :3] = r @ global_cov[3:, :3] @ r.T
            ret[:3, 3:] = r @ global_cov[:3, 3:] @ r.T
            ret[3:, 3:] = r @ global_cov[3:, 3:] @ r.T
            return ret
        else:
            raise Exception(f"Unsupported covariance shape {global_cov.shape}")

    #def transfrom_
    
    @staticmethod
    def to_rotation(vec: ArrayLike) -> Rotation:
        """
        Derives a quaternion for a coordinate system: 
        Forward=[1,0,0], Left=[0,1,0], Up=[0,0,1].
        The result will have zero roll relative to the XY plane.
        """
        vec = np.asarray(vec)
        
        # 1. Normalize the target forward vector (Local X)
        forward = vec / np.linalg.norm(vec)
        
        # 2. Derive the Local Left vector (Local Y)
        # Crossing Global Up with Forward gives a vector on the XY plane
        left = np.cross(Frame.UP, forward)
        
        # Handle singularity: if target is straight up/down, cross product is zero
        if np.linalg.norm(left) < 1e-6:
            # Default to global Left [0, 1, 0] to resolve ambiguity
            left = np.array([0, 1, 0])
        else:
            left /= np.linalg.norm(left)
        
        # 3. Derive the Re-orthogonalized Local Up vector (Local Z)
        up = np.cross(forward, left)
        
        # 4. Construct Rotation Matrix
        # Columns correspond to Local X (Forward), Local Y (Left), and Local Z (Up)
        matrix = np.column_stack((forward, left, up))
    
        # 5. Convert to Quaternion [w, x, y, z]
        return Rotation.from_matrix(matrix)

class Point3D:
    # measurement coordinates are (0,0,range)
    range: float    
    frame: Frame

    x_stdev: float
    y_stdev: float
    z_stdev: float
    
    def __init__(self, vec: ArrayLike, forward=[1, 0, 0]):
        vec = np.asarray(vec)
        rng = np.linalg.norm(vec)
        self.range = float(rng)
        self.frame = Frame([0, 0, 0], vec)

        self.x_stdev = self.y_stdev = self.z_stdev = 1

    def __repr__(self) -> str:
        return f"Point3D(range={self.range:.2f}, frame={self.frame})"

In [5]:
# identity
f = Frame([0, 0, 0], [1, 0, 0])
assert np.allclose(f.to_local_vec([1, 0, 0]), [1, 0, 0], atol=.01)

# -45 deg az
f = Frame([0, 0, 0], [1, 1, 0])
assert np.allclose(f.to_local_vec([1, 0, 0]), [.7, -.7, 0], atol=.01)

# -90 deg az
f = Frame([0, 0, 0], [0, 1, 0])
assert np.allclose(f.to_local_vec([1, 0, 0]), [0, -1, 0], atol=.01)

# identity
f = Frame([0, 0, 0], [1, 1, 0])
assert np.allclose(f.to_local_vec([1, 1, 0]), [1.41, 0, 0], atol=.01)

# 45 deg down
f = Frame([0, 0, 0], [1, 1, 1.4142])
assert np.allclose(f.to_local_vec([1, 1, 0]), [1, 0, -1], atol=.01)

In [6]:
f = Frame([0, 0, 0], [1.732, 1, 0])

# 30 degrees left
e = f.from_local_direction([1.732, 1, 0]).rotation.as_euler('xyz', degrees=True)
assert np.allclose(e, [0, 0, 60], atol=.01), f"{e}"

# 30 degrees up
e = f.from_local_direction([1.732, 0, 1]).rotation.as_euler('xyz', degrees=True)
assert np.allclose(e, [0, -30, 30], atol=.01), f"{e}"

# 30 degrees up, 30 degrees left
e = f.from_local_direction([1.732, 1, 2/np.sqrt(3)]).rotation.as_euler('xyz', degrees=True)
assert np.allclose(e, [0, -30, 60], atol=.01), f"{e}"

# 30 deg up
f = Frame([0, 0, 0], [1.732, 0, 1])

# 30 degrees up
e = f.from_local_direction([1.732, 0, 1]).rotation.as_euler('xyz', degrees=True)
assert np.allclose(e, [0, -60, 0], atol=.01), f"{e}"

In [7]:
p = Point3D([1, 1, 1])
p

Point3D(range=1.73, frame=Frame(origin=[0, 0, 0], rotation=(45.00, -35.26, 0.00)))

## Track in 3D

In [15]:
class Tracker3D:
    """Position + velocity"""
    
    def __init__(self):
        self.x     = np.zeros((6, 1))
        self.P     = np.eye(6)
        self.frame = Frame([0, 0, 0], [1, 0, 0])

        self.H     = np.asarray([[1, 0, 0, 0, 0, 0],
                                 [0, 1, 0, 0, 0, 0],
                                 [0, 0, 1, 0, 0, 0]])
        self.motion = ConstantVelocityModel()
        
        self.v = None
        self.S = None
        self.K = None

    def initialize(self, x: ArrayLike):
        x = np.asarray(x).flatten()
        self.x[:len(x), 0] = x
    
    def predict(self, dt: float):
        F = self.motion.F(dt)
        Q = self.motion.Q(dt)
        
        self.x = F @ self.x
        self.P = F @ self.P @ F.T + Q
    
    def update(self, meas: Point3D):
        # TODO support different origins
        assert np.allclose(meas.frame.origin, self.frame.origin)
        
        # 1. express measurement in the track frame
        delta_rot = self.frame.rotation.inv() * meas.frame.rotation
        delta_frame = Frame(self.frame.origin, delta_rot)

        # delta_frame transforms from measurement frame to track frame;
        # z starts in measurement frame (range-only) and ends in the track frame
        z = np.asarray([meas.range, 0, 0])
        z = delta_frame.to_global_vec(z)
        
        z = z.reshape((3, 1)) # column vector

        R = np.diag([meas.x_stdev, meas.y_stdev, meas.z_stdev])
        R = delta_frame.to_local_cov(R)
        
        # 2. calculate innovation
        self.S = self.H @ self.P @ self.H.T + R
        self.v = z - self.H @ self.x

        # 3. Kalman gain
        self.K = self.P @ self.H.T @ np.linalg.inv(self.S)

        # 4. filtered mean + cov
        self.x = self.x + self.K @ self.v
        self.P = self.P - self.K @ self.S @ self.K.T

        # 5. global frame centered at the new target position
        new_frame = self.frame.from_local_direction(self.x[:3, 0])

        # 6. delta rotation, delta frame; transforms from the current track frame
        #    to the new track frame
        delta_rot   = new_frame.rotation * self.frame.rotation.inv()
        delta_frame = Frame(np.zeros(3), delta_rot)

        # 7. rotate state to the new frame
        self.P  = delta_frame.to_local_cov(self.P)

        # rotate position and velocity separately
        self.x = delta_frame.to_local_vec(self.x).reshape((-1, 1))

        self.frame = new_frame

In [16]:
t = Tracker3D()
t.initialize([10, 0, 0])
t.predict(1)

t.update(Point3D([10, 10, 0]))

In [17]:
t.x

array([[12.20655562],
       [ 0.        ],
       [ 0.        ],
       [ 2.58058055],
       [ 3.68654364],
       [ 0.        ]])

In [18]:
print_rot(t.frame.rotation)

yaw: 34.99, pitch: 0.00, roll: 0.00


In [19]:
t.frame.to_global_vec(t.x[:3,0])

array([10.,  7.,  0.])

In [20]:
with np.printoptions(precision=2, suppress=True):
    print(t.P)

[[ 0.7  -0.    0.    0.45 -0.    0.  ]
 [-0.    0.7   0.   -0.    0.45  0.  ]
 [ 0.    0.    0.7   0.    0.    0.45]
 [ 0.45  0.    0.    1.32 -0.    0.  ]
 [ 0.    0.45  0.   -0.    1.32  0.  ]
 [ 0.    0.    0.45  0.    0.    1.33]]


In [21]:
meas = Point3D([10, 10, 0])
frame = Frame([0, 0, 0], [10, 0, 0])

In [22]:
# 1. express measurement in the track frame
delta_rot = frame.rotation.inv() * meas.frame.rotation
delta_frame = Frame(frame.origin, delta_rot)

print_rot(delta_rot)

yaw: 45.00, pitch: 0.00, roll: 0.00


In [23]:
z = np.asarray([meas.range, 0, 0])
z = delta_frame.to_global_vec(z)
z

array([10., 10.,  0.])

In [24]:
meas.frame.to_global_vec([meas.range, 0, 0])

array([10., 10.,  0.])